In [ ]:
!pip3 install  -r requirements.txt

In [1]:
from dotenv import dotenv_values
import openai, os
import numpy as np
from numpy.linalg import norm

secrets= dotenv_values(".env")

In [2]:
os.environ['OPENAI_API_KEY'] = secrets['OPENAI_API_KEY']

In [4]:
from unstructured.partition.pdf import partition_pdf

path = "data/"

raw_pdf_elements = partition_pdf(
    filename=path + "kafka_paper.pdf",
    extract_images_in_pdf=True,
    infer_table_structure=True,
    chunking_strategy="by_title",
    max_characters=1000, 
    new_after_n_chars=800, 
    combine_text_under_n_chars=500,
    image_output_dir_path=path + 'images/'
)

In [8]:
1000*56

56000

In [5]:
raw_pdf_elements[1].text 

'General Terms\n\ngranular click-through rates, which generate log records not only for every user click, but also for dozens of items on each page that are not clicked. Every day, China Mobile collects 5-8TB of phone call records [11] and Facebook gathers almost 6TB of various user activity events [12].\n\nMany early systems for processing this kind of data relied on physically scraping log files off production servers for analysis. In recent years, several specialized distributed log aggregators have been built, including Facebook’s Scribe [6], Yahoo’s Data Highway [4], and Cloudera’s Flume [3]. Those systems are primarily designed for collecting and loading the log data into a data warehouse or Hadoop [8] for offline consumption. At LinkedIn (a social network site), we found that in addition to traditional offline analytics, we needed to support most of the real-time applications mentioned above with delays of no more than a few seconds.'

In [22]:
raw_pdf_elements

In [9]:
table_elements = []
text_elements = []
for element in raw_pdf_elements:
    if element.category == "Table":
        table_elements.append(element.text)
    elif element.category == "CompositeElement":
        text_elements.append(element.text)


table_elements = list(filter(None, table_elements))
text_elements = list(filter(None, text_elements))
print(len(table_elements))
print(len(text_elements))

0
56


In [7]:
text_elements

['Kafka: a Distributed Messaging System for Log Processing\n\nJay Kreps LinkedIn Corp. jkreps@linkedin.com\n\nNeha Narkhede LinkedIn Corp. nnarkhede @linkedin.com\n\nJun Rao LinkedIn Corp. jrao@linkedin.com\n\nABSTRACT\n\nLog processing has become a critical component of the data pipeline for consumer internet companies. We introduce Kafka, a distributed messaging system that we developed for collecting and delivering high volumes of log data with low latency. Our system incorporates ideas from existing log aggregators and messaging systems, and is suitable for both offline and online message consumption. We made quite a few unconventional yet practical design choices in Kafka to make our system efficient and scalable. Our experimental results show that Kafka has superior performance when compared to two popular messaging systems. We have been using Kafka in production for some time and it is processing hundreds of gigabytes of new data each day.',
 'General Terms\n\ngranular click-thr

## Summarize

In [29]:
from langchain.chains import LLMChain
from langchain.output_parsers import NumberedListOutputParser
from langchain.chat_models import ChatOpenAI

prompt = """You are an AI PhD specialized in AI and RAG, you are tasked with summarizing tables and text. 
Give a concise summary of the table or text.

Table or text chunk: {element}
""" 
model_name='gpt-4' #"text-embedding-3-small" 
model = ChatOpenAI(temperature=0, model_name=model_name)
summarize_chain = LLMChain.from_string(
    llm=model,
    template=prompt
)

In [36]:
summarize_chain.batch(table_elements)


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4 in organization org-ohCyPKiaZocqkltV3f3wdjGH on tokens per min (TPM): Limit 10000, Used 9962, Requested 71. Please try again in 198ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}